In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
%matplotlib inline

In [ ]:
df_1 = pd.read_csv(r'/Users/eliskasimova/Desktop/data_analytics_course_2024/project_folder/labs/second_project/data/clean/df_merged.csv')
df_2 = pd.read_csv(r'/Users/eliskasimova/Desktop/data_analytics_course_2024/project_folder/labs/second_project/data/raw/df_final_web_data_pt_1.txt')
df_3 = pd.read_csv(r'/Users/eliskasimova/Desktop/data_analytics_course_2024/project_folder/labs/second_project/data/raw/df_final_web_data_pt_2.txt')
merged_df = pd.concat([df_2, df_3], axis=0)

df_merged = df_1.merge(merged_df, on='client_id', how='inner')
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'], errors='coerce')  # coerce invalid formats to NaT

### Groups sorting to control/test

In [ ]:
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

# Sort control group
control_group_sorted = control_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])

# Sort test group
test_group_sorted = test_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])

In [ ]:
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'], errors='coerce')  # Coerce invalid formats to NaT
# Filter for the test group only
control_group = df_merged[df_merged['variation'] == 'Control']
# Sort by client_id and date_time to ensure chronological order of events
control_group = control_group.sort_values(by=['client_id', 'date_time'])
# Function to calculate completion time for each step in the test group
def calculate_completion_time(group_df):
    # Create a new column to store the completion time for each step
    group_df['next_step_time'] = group_df.groupby('client_id')['date_time'].shift(-1)
    # Only keep rows where the next step exists (i.e., not NaT)
    group_df = group_df.dropna(subset=['next_step_time'])
    # Calculate the completion time as the time difference between current and next step
    group_df['completion_time'] = group_df['next_step_time'] - group_df['date_time']
    return group_df[['client_id', 'process_step', 'date_time', 'next_step_time', 'completion_time']]
# Apply the function to the test group
control_group_completion_times = calculate_completion_time(control_group)

# Convert completion_time to minutes for easier interpretation
control_group_completion_times['completion_time_minutes'] = control_group_completion_times['completion_time'].dt.total_seconds() / 60

# Calculate the IQR (Interquartile Range) for completion time
Q1 = control_group_completion_times['completion_time_minutes'].quantile(0.25)
Q3 = control_group_completion_times['completion_time_minutes'].quantile(0.75)
IQR = Q3 - Q1

# Define the upper and lower bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers based on IQR
filtered_data = control_group_completion_times[
    (control_group_completion_times['completion_time_minutes'] >= lower_bound) &
    (control_group_completion_times['completion_time_minutes'] <= upper_bound)
]

# Now calculate the average completion time in minutes for each process step, after removing outliers
average_completion_time_minutes_filtered = filtered_data.groupby('process_step')['completion_time_minutes'].mean().reset_index()

# Display the result
print(average_completion_time_minutes_filtered)


### Completion time without outliers for control group per each step

### Completion time analysis with outliers

In [ ]:
# filter to get the latest start for each client
starts_only = df_merged[df_merged['process_step'] == 'start']
latest_starts = starts_only.loc[starts_only.groupby('client_id')['date_time'].idxmax()]

# filter to get the last confirmation for each client
confirmation_only = df_merged[df_merged['process_step'] == 'confirm']
latest_confirms = confirmation_only.loc[confirmation_only.groupby('client_id')['date_time'].idxmax()]

# merge to have both latest start and confirm per client
# Confirming datetime columns are of Timestamp type in merged DataFrame
latest_start_confirms = pd.merge(latest_starts, latest_confirms, on='client_id', suffixes=('_start', '_confirm'))

# calculate process duration for those who completed the process
latest_start_confirms['process_duration'] = latest_start_confirms['date_time_confirm'] - latest_start_confirms['date_time_start']

# Scalculate the average duration and compare with mode and mean
print("Average duration:", latest_start_confirms['process_duration'].mean())
print("Duration mode:", latest_start_confirms['process_duration'].mode())
print("Duration median:", latest_start_confirms['process_duration'].median())


### Completion time analysis without outliers in general (not divided by A/B)

In [ ]:
# Convert the timedelta to seconds for easier manipulation
latest_start_confirms['process_duration_seconds'] = latest_start_confirms['process_duration'].dt.total_seconds()

# Calculate the IQR (Interquartile Range)
Q1 = latest_start_confirms['process_duration_seconds'].quantile(0.25)
Q3 = latest_start_confirms['process_duration_seconds'].quantile(0.75)
IQR = Q3 - Q1

# Define the upper and lower bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers based on IQR
filtered_data = latest_start_confirms[(latest_start_confirms['process_duration_seconds'] >= lower_bound) &
                                      (latest_start_confirms['process_duration_seconds'] <= upper_bound)]

# Convert process_duration back to Timedelta
filtered_data['process_duration'] = pd.to_timedelta(filtered_data['process_duration_seconds'], unit='s')

# Calculate the average process duration again after removing outliers
print("Average duration in total for both groups without outliers:", filtered_data['process_duration'].mean())
print("Median duration in total for both groups without outliers:", filtered_data['process_duration'].median())


### Adding age groups

In [ ]:
# define age bins and categorize ages
bins = [0, 30, 40, 50, 100] # You can decide the intervals based on your data
labels = ['Under 30', '30-39', '40-49', '50 and above']
df_merged['age_group'] = pd.cut(df_merged['clnt_age'], bins=bins, labels=labels)